## Setup

In [1]:
import json

with open("output.json") as f:
    data = json.load(f)

In [2]:
coap = []
for i in range(len(data)):
    if ('coap' in data[i]["_source"]['layers']):
        coap.append(data[i])
print(len(coap))

2584


In [3]:
mqtt = []
for i in range(len(data)):
    if ('mqtt' in data[i]["_source"]['layers']):
        mqtt.append(data[i])
print(len(mqtt))

7711


## Question 1

**How many CoAP GET requests are directed to non-existing resources in the local CoAP server? How many of these requests
are of type Non confirmable?**

Let's start finding the coap GET requests that returned not found: first we retrieve all the coap GET requests, and all the ones that returned not found. The idea, is to see which has the same token or message id (it would mean that one is the answer of the other).

Notice that it would not be enough to check the token!

In [78]:

coapget = []
for i in range(len(coap)):
    if (coap[i]["_source"]['layers']['coap']['coap.code'] == '1'):
        coapget.append(coap[i])
print(f"Number of GET requests: {len(coapget)}")
print(f"Example of token taken from the first coap GET request: {coapget[0]['_source']['layers']['coap']['coap.token']}")

notfoud = []
for i in range(len(coap)):
    if (coap[i]["_source"]['layers']['coap']['coap.code'] == '132'):
        notfoud.append(coap[i])
print(f"Coap requests that returned not found: {len(notfoud)}")
print("Which requests returned not found:")
print(notfoud)

Number of GET requests: 715
Example of token taken from the first coap GET request: 4f:47:aa:e9:8c:78:97:a1
Coap requests that returned not found: 41
Which requests returned not found:
[{'_index': 'packets-2023-03-14', '_type': 'doc', '_score': None, '_source': {'layers': {'frame': {'frame.encap_type': '25', 'frame.time': 'Mar 14, 2023 15:13:02.857578000 CET', 'frame.offset_shift': '0.000000000', 'frame.time_epoch': '1678803182.857578000', 'frame.time_delta': '0.000008000', 'frame.time_delta_displayed': '0.000008000', 'frame.time_relative': '568.861295000', 'frame.number': '9515', 'frame.len': '68', 'frame.cap_len': '68', 'frame.marked': '0', 'frame.ignored': '0', 'frame.protocols': 'sll:ethertype:ip:udp:coap:data-text-lines'}, 'sll': {'sll.pkttype': '0', 'sll.hatype': '1', 'sll.halen': '6', 'sll.src.eth': '52:54:00:12:35:02', 'sll.unused': '0b:2b', 'sll.etype': '0x00000800'}, 'ip': {'ip.version': '4', 'ip.hdr_len': '20', 'ip.dsfield': '0x00000000', 'ip.dsfield_tree': {'ip.dsfield.dscp

Now we need to find which response "not found" is sent from the server, which sends the "not found" message:

In [79]:
notfoundip = []
for i in range(len(notfoud)):
    if (notfoud[i]["_source"]['layers']['ip']['ip.src'] == '127.0.0.1'):
        notfoundip.append(notfoud[i])
print(f"Number of not found sent from the server: {len(notfoundip)}")

Number of not found sent from the server: 38


Now we have to check which, among those, are originated from a GET request

In [80]:
out = [] # not found responses to a get request
outget = [] # get responses that returned not found
notout = [] # not found responses to a not get request
notoutget = [] # not get requests that returned not found

for i in range(len(coapget)): # for each get request
    for j in range(len(notfoundip)): # for each response that returned "not found"
        try: # If they have a token
            if (coapget[i]["_source"]['layers']['coap']['coap.token'] == notfoundip[j]["_source"]['layers']['coap']['coap.token']):
                out.append(notfoundip[j])
                outget.append(coapget[i])
            else:
                notout.append(notfoundip[j])
                notoutget.append(coapget[i])
        except: # if they don't have a token, it will raise an exception. We can check the message id
            if (coapget[i]["_source"]['layers']['coap']['coap.mid'] == notfoundip[j]["_source"]['layers']['coap']['coap.mid']):
                out.append(notfoundip[j])
                outget.append(coapget[i])
            else:
                notout.append(notfoundip[j])
                notoutget.append(coapget[i])
print(f"Number of GET requests which returned a \"not found\": {len(out)}")

Number of GET requests which returned a "not found": 11


Now we can try to build a filter to confirm what we saw here on wireshark:

In [81]:
filterout = ""
filteroutget = ""
filternotout = ""
filternotoutget = ""

for x in out:
    filterout += 'frame.number == ' + x["_source"]['layers']['frame']['frame.number'] + ' || '
print(filterout)
for x in outget:
    filteroutget += 'frame.number == ' + x["_source"]['layers']['frame']['frame.number'] + ' || '
print(filteroutget)
for x in notout:
    filternotout += 'frame.number == ' + x["_source"]['layers']['frame']['frame.number'] + ' || '
print(filternotout)
for x in notoutget:
    filternotoutget += 'frame.number == ' + x["_source"]['layers']['frame']['frame.number'] + ' || '
print(filternotoutget)

frame.number == 32459 || frame.number == 32886 || frame.number == 33131 || frame.number == 33653 || frame.number == 34302 || frame.number == 34477 || frame.number == 34662 || frame.number == 36646 || frame.number == 37411 || frame.number == 37475 || frame.number == 39190 || 
frame.number == 32458 || frame.number == 32885 || frame.number == 33130 || frame.number == 33652 || frame.number == 34301 || frame.number == 34476 || frame.number == 34661 || frame.number == 36645 || frame.number == 37410 || frame.number == 37474 || frame.number == 39189 || 
frame.number == 32459 || frame.number == 32785 || frame.number == 32886 || frame.number == 33131 || frame.number == 33653 || frame.number == 33775 || frame.number == 33795 || frame.number == 33881 || frame.number == 34302 || frame.number == 34475 || frame.number == 34477 || frame.number == 34516 || frame.number == 34524 || frame.number == 34538 || frame.number == 34662 || frame.number == 34664 || frame.number == 35009 || frame.number == 35011 |

As we can see, to confirm what we said before, some of them does not have a token. They have been matched through the message id.

- 32459
- 32886
- 33131
- 33653
- 34302
- 34477
- 34662
- 36646 -> this number does not have a token
- 37411
- 37475
- 39190 -> this number does not have a token

To answer to the second part of the question, and see which of them are non confirmable, we just need to filter on the `type` of the message. We can simply start from the array we found before, and check for the non confirmable type.

In [82]:
out 
nonconf = []
for x in out:
    if (x["_source"]['layers']['coap']['coap.type'] == '1'):
        nonconf.append(x)
print(f"Among the messages found before, {len(nonconf)} were non confirmable")

Among the messages found before, 6 were non confirmable


## Question 2
**How many CoAP DELETE requests directed to the `coap.me` server did not produce a successful result? How many of these are
directed to the `/hello` resource?**


### resolve `coap.me`

On wireshark filter for dns and check for a response for the domain `coap.me`

```coap.me: type A, class IN, addr 134.102.218.18```

### part 1

We need to find how many coap delete requests directed to that server did not produce a successful result. We can start by searching the delete requests

In [83]:
delete = []
for x in coap:
    if (x["_source"]['layers']['coap']['coap.code'] == '4') and (x['_source']['layers']['ip']['ip.dst'] == '134.102.218.18'):
        delete.append(x)
print(f"The server 134.102.218.18 received {len(delete)} delete requests")

The server 134.102.218.18 received 115 delete requests


We are asked to find the ones that produced a successful result. It is enough to check those that did not produced a good result (which in case of a delete request, are those that didn't returned a `deleted` result)

Let's find then the number of requests that produced a good result

In [84]:
goodresp = []
for x in coap:
    if (int(x["_source"]['layers']['coap']['coap.code']) ==66 and x['_source']['layers']['ip']['ip.src'] == '134.102.218.18'):
        goodresp.append(x)
print(f"The delete requests that produced a good result were {len(goodresp)}")
print(f"So, over {len(delete)} delete requests, the unsuccesful ones were {(len(delete) - len(goodresp))}")

The delete requests that produced a good result were 10
So, over 115 delete requests, the unsuccesful ones were 105


### part 2


Now we are asked to find, among the unsuccessful ones, the one that were directed to the `/hello` resource.

Before we just found the number of unsuccessful ones, but we did not identify them. Now we need to find if a request was successful. If it was not, then we need to add it to our list of requests that were bad.

Notice that we can proceed as before, working with tokens and message ids.

In [85]:
deletebad = []
for x in delete:
    flag = False
    try:
        for y in goodresp:
            if (x["_source"]['layers']['coap']['coap.token'] == y["_source"]['layers']['coap']['coap.token']):
                flag = True
    except:
        if (x["_source"]['layers']['coap']['coap.mid'] == y["_source"]['layers']['coap']['coap.mid']):
            flag = True
    if (flag == False):
        deletebad.append(x)
print(f"This is just a check. We need to obtain the previous number. Delete requests with bad result: {len(deletebad)}")

This is just a check. We need to obtain the previous number. Delete requests with bad result: 105


Now that we identified which requests produced a bad result, we can find which of those were directed to the `/hello` resource.

In [86]:
deletebadhello = []
for x in deletebad:
    if (x['_source']['layers']['coap']['coap.opt.uri_path_recon'] == '/hello'):
        deletebadhello.append(x)
print(f"Number of unsuccessful delete requests directed to the \"\hello\" resource: {len(deletebadhello)}")

Number of unsuccessful delete requests directed to the "\hello" resource: 5


## Question 3

**How many different MQTT clients subscribe to the public broker mosquitto using single-level (`+`) wildcards? How many of these clients WOULD receive a publish message issued to the topic `hospital/room2/area0`**

On Wireshark, filter for dns and check for a response for the domain `test.mosquitto.org`. An example of answer, taken from the file, is:

```test.mosquitto.org: type A, class IN, addr 91.121.93.94```

Let's print an example of topic, to see if we are accessing correctly the json topic:

In [87]:
print(mqtt[450]['_source']['layers']['mqtt']['mqtt.topic'])

metaverse/facility2/room6/humidity


Now, having found the ip, we can check the subscribe messages to that ip, whose topic contain a `+`. Later we will match the topic to the requested one. Since they are few, we can also print them easily.

In [88]:
mqip = '91.121.93.94'
ff = '' # Will be needed later, to print the filter.
subbed = []
for x in mqtt:
    if (x['_source']['layers']['mqtt']['mqtt.hdrflags_tree']['mqtt.msgtype'] == '8' and x['_source']['layers']['ip']['ip.dst'] == mqip) and ('+' in x['_source']['layers']['mqtt']['mqtt.topic']):
        subbed.append(x)
        ff += 'frame.number == ' + x["_source"]['layers']['frame']['frame.number'] + ' || '
        print(x['_source']['layers']['mqtt']['mqtt.topic'])

university/+/room3
house/room2/+/light
house/building3/section2/+
university/+/#
hospital/+/area0
house/department1/+
university/+
university/+/floor6/light
metaverse/facility4/+/temperature
metaverse/+/room3
hospital/room2/+/+
house/+/room3
metaverse/+/#


Now we found the subscriptions to the `mosquitto` with single level wildcard. We need to understand wether they are single clients or not. How to do that? We decided to check the source port of the connection.

Below an example of how to access it

In [89]:
subbed[0]['_source']['layers']['tcp']['tcp.srcport']

'51531'

Now we need to map them and see if they are single or not.

In [90]:
singleclient = {}
for x in subbed:
    if (x['_source']['layers']['tcp']['tcp.srcport'] in singleclient.keys()):
        singleclient[x['_source']['layers']['tcp']['tcp.srcport']].append(x)
    else:
        singleclient[x['_source']['layers']['tcp']['tcp.srcport']] = [x]
print(f"Number of single clients that subscribed with a single layer wildcard (+): {len(singleclient)}")

Number of single clients that subscribed with a single layer wildcard (+): 3


Now we need to answer to the second part of the question. we have two possible interpretations:
1. Which single client through the subscription with a single level wildcard actually match `hospital/room2/area0`?
2. Which single client through also other subscription will actually receive messages in `hospital/room2/area0`?

### Interpretation 1

To see which would receive a publish message issued to the topic `hospital/room2/area0`, we could in principle check the above results manually, since they are not a lot, but we opted for a regex, in order to make the process automatic.

In [91]:
import re

pattern = '^(#|((\+|hospital)\/(#|((\+|room2)\/(\+|area0|#)))))$'

matching_subscriptions = []
for client in singleclient.values():
    for subscription in client:
        if re.match(pattern, subscription['_source']['layers']['mqtt']['mqtt.topic']):
            matching_subscriptions.append(x)
            break

print(f"Of those, {len(matching_subscriptions)} would receive a message issued to the topic \"hospital/room2/area0\"")


Of those, 1 would receive a message issued to the topic "hospital/room2/area0"


### Interpretation 2

In this case, we need to check the clients who made a subscription to the public broker `mosquitto` to whatever topic through a single level wildgard (`+`), and among those, check who subscribed to the same server in a way that matches `hospital/room2/area0` (also counting `+` and `#` wildcards).

We already have the list of single clients that subbed with a single level wildcard, it's inside the map `singleclient{}`.

It is enough to check all the subscription that those clients made to the `mosquitto` broker, and see whether they match the string in the regex `pattern`

In [92]:
subscriptions_for_client_to_mosquitto = {} # subscriptions for each single client to the server mosquitto
pattern = '^(#|((\+|hospital)\/(#|((\+|room2)\/(\+|area0|#)))))$'

# building the list of subscriptions per single client found before to the mosquitto server
for client in singleclient.values():
    for mqtt_message in mqtt:
        # if is a subscription to the server
        if (mqtt_message['_source']['layers']['mqtt']['mqtt.hdrflags_tree']['mqtt.msgtype'] == '8' and mqtt_message['_source']['layers']['ip']['ip.dst'] == mqip):
            if (mqtt_message['_source']['layers']['tcp']['tcp.srcport'] in subscriptions_for_client_to_mosquitto.keys()):
                subscriptions_for_client_to_mosquitto[mqtt_message['_source']['layers']['tcp']['tcp.srcport']].append(mqtt_message)
            else:
                subscriptions_for_client_to_mosquitto[mqtt_message['_source']['layers']['tcp']['tcp.srcport']] = [mqtt_message]


# checking whether something matches
matching_subscriptions_v2 = []
for client in subscriptions_for_client_to_mosquitto.values():
    for subscription in client:
        if re.match(pattern, subscription['_source']['layers']['mqtt']['mqtt.topic']):
            matching_subscriptions_v2.append(x)
            break

print(f"Of those, {len(matching_subscriptions_v2)} would receive a message issued to the topic \"hospital/room2/area0\"")

Of those, 2 would receive a message issued to the topic "hospital/room2/area0"


## Question 4

**How many MQTT clients specify a last Will Message directed to a topic having as first level `university`? How many of these Will
Messages are sent from the broker to the subscribers?**

The last will is set on connection. So, we need to check, among mqtt connection, who set the last will message

In [30]:
connect = []
for x in mqtt:
    if (x['_source']['layers']['mqtt']['mqtt.hdrflags_tree']['mqtt.msgtype'] == '1'):
        if 'mqtt.willmsg' in (x['_source']['layers']['mqtt']):
            connect.append(x)
print(f"Number of devices that connected with last will property and message set: {len(connect)}")

Number of devices that connected with last will property and message set: 7


To see which of those set a last will having at first level university, we just need to take the first level of the `willtopic`.

In [31]:
connectuni = []
for x in connect:
    if 'university' == x['_source']['layers']['mqtt']['mqtt.willtopic'].split('/')[0]:
        connectuni.append(x)
print(f"Number of devices with last will property and message set and directed to a topic having as first level \"university\": {len(connectuni)}")

Number of devices with last will property and message set and directed to a topic having as first level "university": 3


Since they are not a lot, we decided to analyse them and print the message, by decoding it:

In [32]:
for x in connectuni:
    print(bytes.fromhex(x['_source']['layers']['mqtt']['mqtt.willmsg'].replace(':', '')).decode())

error: cdfxqpme
error: frfhvyia
error: yadwjrbj


It is enoug to see if they have been sent (actually, in this case, we can immediately see that no message mqtt containing `error` was sent, so we can already stop here)

In [33]:
willsent = []
for x in mqtt:
    try:
        if ('error' in bytes.fromhex(x['_source']['layers']['mqtt']['mqtt.msg'].replace(':', '')).decode()):
            willsent.append(x)
    except:
        pass
print(f"Number of last will message sent: {len(willsent)}")

Number of last will message sent: 0


## Question 5
How many Publish messages with QoS = 1 are received by the MQTT clients connected to the HiveMQ broker with MQTT
version 5?


- broker.hivemq.com: type A, class IN, addr 52.29.173.150
- broker.hivemq.com: type A, class IN, addr 3.65.137.17

In [17]:
tohive = []
for x in mqtt:
    if (x['_source']['layers']['ip']['ip.dst'] == '52.29.173.150' or x['_source']['layers']['ip']['ip.dst'] == '3.65.137.17'):
        tohive.append(x)
print(f"Number of messages sent to the broker: {len(tohive)}")


# This is the list of all the pub recived ack messages sent to the broker
ack = []
for x in tohive:
    if (x['_source']['layers']['mqtt']['mqtt.hdrflags_tree']['mqtt.msgtype'] == '5' or x['_source']['layers']['mqtt']['mqtt.hdrflags_tree']['mqtt.msgtype'] == '4'):
        ack.append(x)
print(f"Number of ack messages sent: {len(ack)}")

singleclient = {}
for x in ack:
    if (x['_source']['layers']['tcp']['tcp.srcport'] in singleclient.keys()):
        singleclient[x['_source']['layers']['tcp']['tcp.srcport']].append(x)
    else:
        singleclient[x['_source']['layers']['tcp']['tcp.srcport']] = [x]
print(f"Number of single clients that sent ack messages: {len(singleclient)}")

clientv5 = []
for x in mqtt:
    if (x['_source']['layers']['mqtt']['mqtt.hdrflags_tree']['mqtt.msgtype'] == '1'):
        for y in singleclient.values():
            if (x['_source']['layers']['ip']['ip.src'] == y[0]['_source']['layers']['ip']['ip.src'] and x['_source']['layers']['mqtt']['mqtt.ver'] == 5):
                clientv5.append(x)
print(f"Number of single clients that sent ack messages and are using MQTT v5: {len(clientv5)}")



Number of messages sent to the broker: 945
Number of ack messages sent: 98
Number of single clients that sent ack messages: 5
Number of single clients that sent ack messages and are using MQTT v5: 0


In [18]:
tohive[0]['_source']['layers']['mqtt']['mqtt.ver']



'4'

## Question 6

**How many MQTT-SN (port 1885) publish messages sent after the hour 3.16PM (Milan Time) are directed to topic 9? Are these
messages handled by the server?**

We need to filter publish messages of MQTT-SN sent after 3.16 PM directed to topic 9. This corresponds to use as filter in wireshark the following:

```
mqttsn && !icmp && mqttsn.topic.id == 9 && mqttsn.msg.type == 0xc
```

(the `!icmp` is necessary to get actually the mqttsn publish messages and not their response)

In [34]:
mqttsn = []
for x in data:
    try:
        # check that a message is a publish message
        if (x['_source']['layers']['udp']['udp.dstport'] == '1885' and x['_source']['layers']['data']['data.data'].split(':')[1] == '0c'):
            # check that the topyc id is 9
            if((x['_source']['layers']['data']['data.data'].split(':')[3] + x['_source']['layers']['data']['data.data'].split(':')[4]) == '0009'):
                mqttsn.append(x)
    except:
        pass
print(f"Number of publish messages directed to topic id 9: {len(mqttsn)}")

Number of publish messages directed to topic id 9: 28


Now we need to check the constraint on the time. This corresponds to set in wireshark the filter:
```
frame.time_epoch >= 1678803360
```
Where did we find this number? Just write as a timestamp the constraint time. In python this can be done through:
```
datetime.datetime.timestamp(datetime.datetime(2023,3,14,15,16,00))
```

In [35]:
import datetime

mqttsntime = []
for x in mqttsn:
    if (float(x['_source']['layers']['frame']['frame.time_epoch']) >= datetime.datetime.timestamp(datetime.datetime(2023,3,14,15,16,00))):
        #print(datetime.datetime.fromtimestamp(float(x['_source']['layers']['frame']['frame.time_epoch'])).strftime('%Y-%m-%d, %H:%M:%S'))
        mqttsntime.append(x)
print(f"Number of messages that stisfies the time constraint: {len(mqttsntime)}")

Number of messages that stisfies the time constraint: 15
